In [1]:
import pandas as pd
import numpy as np
import polars as pl
import datetime

pd.set_option("display.max_columns", None)

In [2]:
### Target col
[
    "hospcode",
    "hosname",
    "name_th",
    "county",
    "changwatname",
    "ampurname",
    "tambonname",
    "sex",
    "birth",
    "age",
    "nation",
    "typearea",
    "diagcode",
    "diagtype",
    "date_serv",
    "chiefcomp",
    "occupation",
    "diagname",
    "typediag",
]
### Current_col
current_col = [
    "opd.hospcode",
    "c.hosname",
    "opd.pid",
    "opd.seq",
    "p.cid",
    "p.sex",
    "p.birth",
    "age",
    "p.nation",
    "p.typearea",
    "opd.diagcode",
    "opd.diagtype",
    "opd.date_serv",
    "p.occupation_new",
    "occ.occupation_new",
    "p.occupation_old",
    "old.occupation",
    "c.provcode",
    "ch.changwatname",
    "c.distcode",
    "ca.ampurname",
    "c.subdistcode",
    "ct.tambonname",
    "s.instype",
    "s.typeout",
    "s.chiefcomp",
]

In [3]:
occ = pl.read_excel(
    "./utilities/occupations.xlsx", schema_overrides={"code": pl.String}
)

# hoscode = pl.read_csv("./utilities/hospcode.csv", dtypes={"hospcode": str}).select(
#     [
#         "hospcode",
#         "name_th",
#         pl.col("changwatname").alias("province"),
#         pl.col("ampurname").alias("district"),
#         pl.col("tambonname").alias("subdistrict"),
#     ]

hoscode = pl.read_csv("./filled_hos.csv", dtypes={"5_number": str}).select(
    ["5_number", "hosname", "odpc", "province", "district", "subdistrict"]
)

total_pop = (
    pl.read_csv("./utilities/total_pop.csv", dtypes={"year": pl.String})
    .select(["year", "total_pop", "DistrictThaiShort", "ProvinceThai"])
    .group_by(by=["year", "DistrictThaiShort", "ProvinceThai"])
    .agg(pl.col("total_pop").sum())
)


icd10 = pl.read_csv("./utilities/icd10.csv", dtypes={"code": pl.String}).unique()


nation = pl.read_csv("./utilities/nation.csv", dtypes={"code": pl.String}).vstack(
    pl.DataFrame({"code": ["99", "57", "48"], "nation": ["ไทย", "กัมพูชา", "เมียนมา"]})
)


odpc = (
    pl.read_excel("./utilities/Template_เขตสุขภาพ_ภูมิภาค.xlsx")
    .with_columns(pl.col("Regional Health").str.strip_prefix("เขตสุขภาพที่ "))
    .select(
        [pl.col("จังหวัด").alias("province"), pl.col("Regional Health").alias("odpc")]
    )
)

C:\Users\Rawinan\AppData\Local\Temp\ipykernel_9144\761742139.py:14: DeprecationWarning: The argument `dtypes` for `read_csv` is deprecated. It has been renamed to `schema_overrides`.
  hoscode = pl.read_csv("./filled_hos.csv", dtypes={"5_number": str}).select(
C:\Users\Rawinan\AppData\Local\Temp\ipykernel_9144\761742139.py:19: DeprecationWarning: The argument `dtypes` for `read_csv` is deprecated. It has been renamed to `schema_overrides`.
  pl.read_csv("./utilities/total_pop.csv", dtypes={"year": pl.String})
C:\Users\Rawinan\AppData\Local\Temp\ipykernel_9144\761742139.py:21: DeprecationWarning: named `by` param is deprecated; use positional `*args` instead.
  .group_by(by=["year", "DistrictThaiShort", "ProvinceThai"])
C:\Users\Rawinan\AppData\Local\Temp\ipykernel_9144\761742139.py:26: DeprecationWarning: The argument `dtypes` for `read_csv` is deprecated. It has been renamed to `schema_overrides`.
  icd10 = pl.read_csv("./utilities/icd10.csv", dtypes={"code": pl.String}).unique()
C:\U

In [4]:
filtered_hos = hoscode.filter(hoscode["5_number"] == "00935")
filtered_hos

5_number,hosname,odpc,province,district,subdistrict
str,str,str,str,str,str
"""00935""","""สถานีอนามัยเฉลิมพระเกียรติ 60 …","""เขตสุขภาพที่ 6""","""สมุทรปราการ""","""เมืองสมุทรปราการ""","""บางเมือง"""


In [5]:
# data = (
#     pl.read_csv(
#         "I21_I22_I24_OPD_Y2024M01-05_20240620.csv",
#         has_header=False,
#         new_columns=current_col,
#         dtypes={
#             "column_1": str,
#             "column_4": str,
#             "column_24": str,
#             "column_3": str,
#             "column_14": str,
#             "column_12": str,
#             "column_25": str,
#             "column_9": str,
#         },
#     )
#     .drop_nulls(subset=["p.cid"])
#     .drop(
#         columns=[
#             "c.hosname",
#             "c.provcode",
#             "ch.changwatname",
#             "c.distcode",
#             "ca.ampurname",
#             "c.subdistcode",
#             "ct.tambonname",
#         ]
#     )
#     .filter(
#         (~pl.col("s.chiefcomp").str.contains("รับยา|ตามนัด|F/U|ต่อยา|มาตามนัด"))
#         | (
#             (pl.col("s.chiefcomp").str.contains("รับยา|ตามนัด|F/U|ต่อยา"))
#             & pl.col("s.chiefcomp").str.contains("มีอาการ")
#         )
#     )
#     .with_columns(
#         zfill=pl.col("opd.hospcode").cast(pl.String).str.zfill(5).alias("opd.hospcode")
#     )
# )

# # แสดงผลลัพธ์
# print(data)

In [11]:
data = (
    pl.read_csv(
        "I21_I22_I24_OPD_Y2024M01-05_20240620.csv",
        has_header=False,
        new_columns=current_col,
        dtypes={
            "column_1": str,
            "column_4": str,
            "column_24": str,
            "column_3": str,
            "column_14": str,
            "column_12": str,
            "column_25": str,
            "column_9": str,
        },
    )
    .drop_nulls(subset=["p.cid"])
    .drop(
        columns=[
            "c.hosname",
            "c.provcode",
            "ch.changwatname",
            "c.distcode",
            "ca.ampurname",
            "c.subdistcode",
            "ct.tambonname",
        ]
    )
    .filter(
        (~pl.col("s.chiefcomp").str.contains("รับยา|ตามนัด|F/U|ต่อยา|มาตามนัด"))
        | (
            (pl.col("s.chiefcomp").str.contains("รับยา|ตามนัด|F/U|ต่อยา"))
            & pl.col("s.chiefcomp").str.contains("มีอาการ")
        )
    )
    .with_columns(
        pl.col("opd.hospcode").cast(pl.String).str.zfill(5).alias("opd.hospcode")
    )
    #     # .with_columns(
    #     #     opd_hospcode=pl.col("opd.hospcode").apply(lambda x: x.zfill(5) if isinstance(x, str) else str(x).zfill(5))
    # )
    .join(hoscode, how="left", left_on="opd.hospcode", right_on="5_number")
)

C:\Users\Rawinan\AppData\Local\Temp\ipykernel_9144\2360789439.py:2: DeprecationWarning: The argument `dtypes` for `read_csv` is deprecated. It has been renamed to `schema_overrides`.
  pl.read_csv(
C:\Users\Rawinan\AppData\Local\Temp\ipykernel_9144\2360789439.py:18: DeprecationWarning: named `columns` param is deprecated; use positional `*args` instead.
  .drop(
C:\Users\Rawinan\AppData\Local\Temp\ipykernel_9144\2360789439.py:44: DeprecationWarning: The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.
  .join(hoscode, how="left", left_on="opd.hospcode", right_on="5_number")


In [13]:
data.filter(pl.col("opd.hospcode").str.starts_with("0"))

opd.hospcode,opd.pid,opd.seq,p.cid,p.sex,p.birth,age,p.nation,p.typearea,opd.diagcode,opd.diagtype,opd.date_serv,p.occupation_new,occ.occupation_new,p.occupation_old,old.occupation,s.instype,s.typeout,s.chiefcomp,hosname,odpc,province,district,subdistrict
str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,i64,str,str,str,str,str,str,str,str,str
"""03149""","""1457""","""451261""","""499468c719fa020a8e51c53f39c09f…",2,"""9/1/1997""",26,"""99""",1,"""I21""","""1""","""4/1/2024""","""9000""","""นักเรียน/นักศึกษา""",900,"""หญิงบริการ""","""4200""","""1""","""แผลที่แผลหน้าท้อง ก่อนมา 6 วัน""","""โรงพยาบาลส่งเสริมสุขภาพตำบลทมอ""","""เขตสุขภาพที่ 9""","""สุรินทร์""","""ปราสาท""","""ทมอ"""
"""09071""","""3674""","""488005""","""3776d275f266f8fc610ec15eec514d…",1,"""8/10/1965""",58,"""99""",1,"""I249""","""4""","""7/5/2024""","""6111""","""ผู้ปลูกพืชไร่และพืชผัก, ทำไร่,…",1,"""เกษตรกรรม""","""100""","""9""","""case TVD S/P PCI with DES DLP""","""โรงพยาบาลส่งเสริมสุขภาพตำบลพรุ…","""เขตสุขภาพที่ 11""","""พังงา""","""เกาะยาว""","""พรุใน"""
"""06342""","""2657""","""523051""","""9788cdf27e763597a41f347424e4ad…",2,"""1/1/1939""",85,"""99""",1,"""I249""","""4""","""15/3/2024""","""9622""","""คนงานรับจ้างทั่วไป""",904,null,"""100""","""1""","""คัดกรองสูงอายุ""","""โรงพยาบาลส่งเสริมสุขภาพตำบลเหม…","""เขตสุขภาพที่ 1""","""แพร่""","""เมืองแพร่""","""เหมืองหม้อ"""
"""01976""","""4242""","""326197""","""e886d5b71e1cbbf8a55c221a504e77…",1,"""13/10/1962""",61,"""99""",1,"""I219""","""4""","""17/4/2024""","""9622""","""คนงานรับจ้างทั่วไป""",2,"""รับจ้าง""","""4200""","""1""","""ทำแผลที่ขาข้างขวา""","""โรงพยาบาลส่งเสริมสุขภาพตำบลบ้า…","""เขตสุขภาพที่ 6""","""ระยอง""","""วังจันทร์""","""พลงตาเอี่ยม"""
"""07699""","""7610""","""592886""","""baa28bedd3dc7f415ac90c82891424…",2,"""23/2/1992""",32,"""99""",1,"""I211""","""1""","""20/3/2024""","""9622""","""คนงานรับจ้างทั่วไป""",403,null,"""100""","""1""","""เจ็บบริเวณหน้าอกข้างขวา เป็นมา…","""โรงพยาบาลส่งเสริมสุขภาพตำบลบ้า…","""เขตสุขภาพที่ 3""","""พิจิตร""","""บึงนาราง""","""ห้วยแก้ว"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""07279""","""5801""","""401321""","""441d599f674dc017d4ce04ec692e92…",1,"""2/1/1951""",73,"""99""",1,"""I21""","""4""","""2/5/2024""","""9622""","""คนงานรับจ้างทั่วไป""",null,null,"""100""","""1""","""เยี่ยมบ้านผู้ป่วยเหนื่อยง่าย จ…","""โรงพยาบาลส่งเสริมสุขภาพตำบลบ้า…","""เขตสุขภาพที่ 2""","""ตาก""","""เมืองตาก""","""แม่ท้อ"""
"""07537""","""1626""","""265832""","""61439cd8add29d5a5c5b598f40b8dc…",2,"""1/1/1940""",84,"""99""",1,"""I249""","""4""","""26/3/2024""","""6111""","""ผู้ปลูกพืชไร่และพืชผัก, ทำไร่,…",502,null,"""100""","""1""","""HT.IHD""","""โรงพยาบาลส่งเสริมสุขภาพตำบลชุม…","""เขตสุขภาพที่ 2""","""พิษณุโลก""","""บางระกำ""","""ชุมแสงสงคราม"""
"""06020""","""1517""","""152501""","""41ee48f48ab1d1cc23d8cd5024a1d8…",2,"""7/12/1962""",61,"""99""",1,"""I214""","""1""","""13/3/2024""","""9622""","""คนงานรับจ้างทั่วไป""",2,"""รับจ้าง""","""100""","""3""","""ขอใบ refer นครพิงค์""","""โรงพยาบาลส่งเสริมสุขภาพตำบลบ้า…","""เขตสุขภาพที่ 1""","""เชียงใหม่""","""สารภี""","""ท่ากว้าง"""


In [ ]:
filtered_df = data.filter(data["opd.hospcode"] == "935")
filtered_df

In [ ]:
data = data.to_pandas()
data.loc[data["opd.hospcode"] == "99782", "name_th"] = (
    "ศูนย์สุขภาพชุมชนโรงพยาบาลสทิงพระ(สทิงพระ)"
)
data.loc[data["opd.hospcode"] == "99782", "province"] = "สงขลา"
data.loc[data["opd.hospcode"] == "99782", "district"] = "สทิงพระ"
data.loc[data["opd.hospcode"] == "99782", "subdistrict"] = "จะทิ้งพระ"

data.loc[data["opd.hospcode"] == "99942", "name_th"] = "ศูนย์สุขภาพชุมชนโรงพยาบาลบึงโขงหลง"
data.loc[data["opd.hospcode"] == "99942", "province"] = "บึงกาฬ"
data.loc[data["opd.hospcode"] == "99942", "district"] = "บึงโขงหลง"
data.loc[data["opd.hospcode"] == "99942", "subdistrict"] = "บึงโขงหลง"

data.loc[data["opd.hospcode"] == "99742", "name_th"] = (
    "หน่วยบริการปฐมภูมิ กลุ่มงานเวชปฏิบัติครอบครัวและชุมชน รพ.บันนังสตา"
)
data.loc[data["opd.hospcode"] == "99742", "province"] = "ยะลา"
data.loc[data["opd.hospcode"] == "99742", "district"] = "บันนังสตา"
data.loc[data["opd.hospcode"] == "99742", "subdistrict"] = "บันนังสตา"

data.loc[data["opd.hospcode"] == "99745", "name_th"] = (
    "ศูนย์สุขภาพชุมชนเมืองเวชกรรม (เมืองยะลา)"
)
data.loc[data["opd.hospcode"] == "99745", "province"] = "ยะลา"
data.loc[data["opd.hospcode"] == "99745", "district"] = "เมือง"
data.loc[data["opd.hospcode"] == "99745", "subdistrict"] = "สะเตง"

data.loc[data["opd.hospcode"] == "99763", "name_th"] = "ศูนย์สุขภาพชุมชนตำบลยามู"
data.loc[data["opd.hospcode"] == "99763", "province"] = "ปัตตานี"
data.loc[data["opd.hospcode"] == "99763", "district"] = "ยะหริ่ง"
data.loc[data["opd.hospcode"] == "99763", "subdistrict"] = "ยามู"


data.loc[data["name_th"].isnull(), "name_th"] = "ไม่ระบุ"
data.loc[data["name_th"].isnull(), "province"] = "ไม่ระบุ"
data.loc[data["name_th"].isnull(), "district"] = "ไม่ระบุ"
data.loc[data["name_th"].isnull(), "subdistrict"] = "ไม่ระบุ"

data.loc[data["opd.hospcode"] == "99874", "name_th"] = "ศุนย์สุขภาพชุมชนโรงพยาบาลยะรัง"
data.loc[data["opd.hospcode"] == "99874", "province"] = "ปัตตานี"
data.loc[data["opd.hospcode"] == "99874", "district"] = "ยะรัง"
data.loc[data["opd.hospcode"] == "99874", "subdistrict"] = "ปิตูมุดี"
data = pl.from_pandas(data)

In [ ]:
data.describe()

In [ ]:
data_2 = (
    data.with_columns(
        pl.when(pl.col("occ.occupation_new").is_null())
        .then(pl.lit("ไม่ระบุ"))
        .otherwise(pl.col("occ.occupation_new"))
        .alias("occ.occupation_new")
    )
    .with_columns(
        pl.when(pl.col("p.sex") == 1)
        .then(pl.lit("ชาย"))
        .when(pl.col("p.sex") == 2)
        .then(pl.lit("หญิง"))
        .otherwise(pl.lit("ไม่ระบุ"))
        .alias("p.sex")
    )
    .with_columns(pl.col("opd.diagcode").str.replace("[.|-]", ""))
    .with_columns(pl.col("opd.diagcode").str.to_uppercase())
    .with_columns(
        pl.when(pl.col("opd.diagtype") == "1")
        .then(pl.lit("principal dx"))
        .when(pl.col("opd.diagtype") == "2")
        .then(pl.lit("co-morbidity"))
        .when(pl.col("opd.diagtype") == "3")
        .then(pl.lit("complication"))
        .when(pl.col("opd.diagtype") == "4")
        .then(pl.lit("other"))
        .when(pl.col("opd.diagtype") == "5")
        .then(pl.lit("external"))
        .when(pl.col("opd.diagtype") == "6")
        .then(pl.lit("additional"))
        .when(pl.col("opd.diagtype") == "0")
        .then(pl.lit("principal dx"))
        .otherwise(pl.lit("morph"))
        .alias("opd.diagtype")
    )
    .with_columns(dx_group=pl.lit("กลุ่มโรคหัวใจและหลอดเลือด"))
    .with_columns(
        pl.col("opd.date_serv")
        .str.to_date("%d/%m/%Y")
        .dt.strftime("%Y-%m-%d")
        .str.to_date()
        .alias("opd.date_serv")
        # pl.col("opd.date_serv")
        # .str.to_date("%Y-%m-%d")
        # .dt.strftime("%d-%m-%Y")
        # .alias("opd.date_serv")
    )
    .join(icd10, how="left", left_on="opd.diagcode", right_on="code")
    .join(nation, how="left", left_on="p.nation", right_on="code")
    .join(odpc, how="left", left_on="province", right_on="province")
    .group_by(["p.cid", "opd.date_serv"])
    .first()
    .select(
        [
            pl.col("opd.date_serv").alias("date_serv"),
            pl.col("odpc"),
            pl.col("province").alias("changwatname"),
            pl.col("district").alias("ampurname"),
            pl.col("subdistrict").alias("tambonname"),
            pl.col("opd.hospcode").alias("hospcode"),
            pl.col("name_th").alias("hosname"),
            pl.col("p.sex").alias("sex"),
            pl.col("p.birth").alias("birth"),
            pl.col("age"),
            pl.col("nation"),
            pl.col("s.chiefcomp").alias("chiefcomp"),
            pl.col("occ.occupation_new").alias("occupation"),
            pl.col("p.typearea").alias("typearea"),
            pl.col("opd.diagcode").alias("diagcode"),
            pl.col("opd.diagtype").alias("diagtype"),
            pl.col("dx_group").alias("typediag"),
            pl.col("dz_name").alias("diagname"),
        ]
    )
    .with_columns(
        pl.when(pl.col("nation").is_null())
        .then(pl.lit("ไม่ระบุ"))
        .otherwise(pl.col("nation"))
        .alias("nation")
    )
    .with_row_count(name="id", offset=1)
    .with_columns(create_date=datetime.date.today())
    .with_columns(
        pl.col("create_date").dt.strftime("%Y-%m-%d").str.to_date().alias("create_date")
    )
)

In [ ]:
data_2.describe()

In [ ]:
data_2.write_csv("I21_I22_I24_01-052024.csv")

In [ ]:
# median_h10 = (
#     combined_h10.select(["opd.diagcode", "opd.date_serv", "province", "district"])
#     .drop_nulls(subset=["province", "district"])
#     .with_columns(
#         week=pl.col("opd.date_serv").str.to_date("%Y-%m-%d").dt.strftime("%U")
#     )
#     .with_columns(
#         year=pl.col("opd.date_serv").str.to_date("%Y-%m-%d").dt.strftime("%Y")
#     )
#     .with_columns(
#         pl.when(pl.col("week") == "00")
#         .then(pl.lit("52"))
#         .otherwise(pl.col("week"))
#         .alias("week")
#     )
#     .drop(columns="opd.date_serv")
#     .group_by(["year", "week", "province", "district"], maintain_order=True)
#     .agg(pl.col("opd.diagcode").alias("no_patients").count())
#     .join(
#         total_pop,
#         how="left",
#         left_on=["province", "district", "year"],
#         right_on=["ProvinceThai", "DistrictThaiShort", "year"],
#     )
#     .filter(~pl.col("total_pop").is_null())
#     .with_columns(
#         rate_per_100000=(pl.col("no_patients") / pl.col("total_pop") * 100000)
#     )
#     .drop(columns=["year", "no_patients", "total_pop"])
#     .group_by(by=["week", "province", "district"])
#     .agg(pl.col("rate_per_100000").median().alias("median_rate_per_100000"))
# )